In [2]:
# using Plots
# using PlotlyJS
# using Statistics
# using QuadGK
# using MAT

## constants

In [ ]:
# physical constants
g = 9.81 # gravitational acceleration [m/s^2]

# fluid properties 
rho_a    = 1.225;     # Density of air [kg/m^3]
c_a      = 1005;      # Specific heat of air [J/KgK]

# building model properties
v_a      = 2880;     # Volume of air inside the building [m^3]
A_f     = 2*90;       # Area of the 3 floors [m^2]
th      = 0.1;       # Thickness of thermal mass floors[m]
E_int    = 250;      # Internal loads due to occupancy, lighting and equipment [W]

# material properties
rho_f   = 2300;     # Density of Thermal Mass concrete [kg/m^3]
k_f     = 0.8;         # Conductivity of Thermal Mass concrete [W/mK]
c_f     = 750;        # Specific heat of Thermal Mass concrete [J/KgK]
h  = 4;           # Convective heat coefficient [W/m2.K]

## functions

In [ ]:

function conductionMatrix(lambda, dt, dx, Nx)
    """calculate temperature evolution within a fabric """
    A = zeros(Nx, Nx)

    A[1,1] = 1-lambda;  
    A[1,2] = lambda;
    A[Nx,Nx-1] = lambda;
    A[Nx,Nx] = 1-lambda;  
    
    # all other rows 
    for i = 2:Nx-1
        A[i, i-1] = lambda;
        A[i, i] = 1-2*lambda;
        A[i, i+1] = lambda;
    end

    return A

end

function cd_pivot(alpha)
    "calculate discharge coefficient based on window opening angle"
    W=1;
    H=1;
    W_pivot = z -> (1/W^2 + 1/(2*(H-z)*tand(alpha)+sind(alpha)*W).^2) .^(-1/2)
    h = H * (1- cosd(alpha));
    integral, est = quadgk(W_pivot,h,H) #rtol=1e-8 -> error when go to last element of angle array
    A_eff = W*h + integral ;
    Cd0 = 0.611;
    Cd = A_eff / (H*W) * Cd0;   
    
end

## initialize variables for convective heat transfer calculation 

In [ ]:
# Discrete variables for 1D conduction calculation over time => T(t+1,x) = A*T(t,x) + b
dt = 15;
dx = th/10;
x = range(0, th, step = dx)  |> collect 
Nx = length(x)

In [ ]:
# initialize A matrix 
alpha = k_f/(rho_f*c_f) # thermal conductivity / (density * specific heat )
lambda = alpha * dt / dx^2 # this could all go into the matrix 
A = conductionMatrix(lambda, dt, dx, Nx);

In [ ]:
# initialize T_f matrix
# T_f => [x * t]

## initialize variables for natural ventilation calculation

In [ ]:
# window properties 
w1 = Dict("alpha" => 42, "area" => 1.61, "l" => 11.34)
w1["cd"] = cd_pivot(w1["alpha"])

w2 = Dict("alpha" => 42, "area" => 1.755, "l" => 6.62)
w2["cd"] = cd_pivot(w2["alpha"])

In [ ]:
# missing variables
time = 0
T_out = 0
T_in = 0

## dynamic thermal model

In [ ]:
for t=1:length(time)-1

    # calculate E_nv at each time step 
    # TODO make function that calculates E_nv, how was this done before we knew outside vs inside?
    for (w, Q) in zip([w1, w2, w3], [Q1, Q2, Q3]) 
        # print("$(T_in[t]-T_out[t]) \n")
        Q[t] = w["cd"] * w["area"] * sqrt(2*g*w["l"] * (T_in[t]-T_out[t]) /(T_out[t]))
    end
    Q_tot[t] = Q1[t] + Q2[t] + Q3[t]
    E_nv[t] = rhoa * ca * Q_tot[t] * (T_out[t] - T_in[t])


    # calculate E_conv at each time step 
    E_conv = h_conv * A_f * (T_f[1,t] - T_in[t]) # TODO multiply by number of surfaces being considered
    E_convst[t] =  E_conv # guessing this is holding the variable?

    # # wall boundary conditions 
    # initialize the b matrix, this likely is related to the boundary conditions somehow..
    b[1] = - lambda * dx / (k_f * E_conv)
    b[Nx] = - lambda * dx / (k_f * E_conv)

    # calculate surface temperature at each time step
    T_f[:, t+1] = A*T_f[:,t] + b'

    # calculate interior average air temperaure at each time step 
    T_in[t + 1] = T_in[t] + (E_conv * A_f + E_in + E_nv[t]) * dt/(v_a * rho_a * c_a)
    Eist[t] = Eint # what does this do///


 end